In [1]:
import chipwhisperer as cw
scope = cw.scope()
scope.gain.db = 25
scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

In [3]:
target = cw.target(scope, cw.targets.CW305, fpga_id="35t", force=False)
# path = "../tragos_victims/vivado_workspace/Artix35t/Artix35t.runs/impl_1/cw305_top.bit"
# target = cw.target(scope, cw.targets.CW305, bsfile=path, force=False)

In [ ]:
target.pll.pll_enable_set(True)
target.pll.pll_outsource_set('PLL0', 0)
target.pll.pll_outfreq_set(10E6,0)

In [4]:
print(target.pll.pll_enable_get())
print(target.pll.pll_outenable_get(0))
print(target.pll.pll_outenable_get(1))
print(target.pll.pll_outenable_get(2))
target.pll.pll_outfreq_get(1)

True
False
True
False


10000000.0

In [5]:
target.vccint_set(1.0)
# we only need PLL1:
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)

# run at 10 MHz:
target.pll.pll_outfreq_set(10E6, 1)

# 1ms is plenty of idling time
target.clkusbautooff = True
target.clksleeptime = 1

In [6]:
if scope._is_husky:
    scope.clock.clkgen_freq = 40e6
    scope.clock.clkgen_src = 'extclk'
    scope.clock.adc_mul = 4
    # if the target PLL frequency is changed, the above must also be changed accordingly
else:
    scope.clock.adc_src = "extclk_x4"

In [7]:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

In [8]:
scope

cwlite Device
sn         = 50203220313038543030393131313039
fw_version = 
    major = 0
    minor = 52
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = False
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 129
    decimate       = 1
    trig_count     = 3030315
    fifo_fill_mode = normal
clock = 
    adc_src       = extclk_x4
    adc_phase     = 0
    adc_freq      = 40001403
    adc_rate      = 40001403.0
    adc_locked    = True
    freq_ctr      = 10000339
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 1
    clkgen_freq   = 192000000.0
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_rx
    tio2         = serial_tx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
   

In [9]:
proj = cw.create_project("passionayui.cwp", overwrite=True)

In [10]:
from tqdm.notebook import trange
N_traces = 5000
ktp = cw.ktp.Basic()
ktp.fixed_key = False
for k in range(10):
    key = ktp.next_key()
text = ktp.next_text()

In [11]:
for i in trange(N_traces):
    trace = cw.capture_trace(scope, target, text, key)
    proj.traces.append(trace)
    text = ktp.next_text()

  0%|          | 0/5000 [00:00<?, ?it/s]

In [12]:
proj.save()
scope.dis()
target.dis()

In [1]:
import chipwhisperer as cw
proj = cw.open_project("passionayui.cwp")

In [2]:
import chipwhisperer.analyzer as cwa 
leak_model = cwa.leakage_models.last_round_state_diff
attack = cwa.cpa(proj, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack), update_interval=1000)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,C00.213,150.203,AC0.220,610.183,DF0.205,610.235,030.209,4D0.194,CC0.199,550.236,AE0.201,3F0.230,210.241,EA0.193,850.193,580.262
1,E10.064,F80.052,190.060,B50.059,D40.068,620.056,370.064,890.053,270.062,760.066,F30.058,690.059,010.068,470.056,0B0.070,920.057
2,530.062,D30.052,E80.060,3E0.054,FC0.063,F40.055,ED0.061,050.053,A80.056,BD0.059,AB0.056,A40.055,090.062,BB0.055,F60.061,6A0.057
3,A20.057,0C0.051,D80.058,AD0.054,790.063,D00.053,8D0.061,300.052,ED0.055,C20.058,300.054,F40.054,650.060,6A0.054,330.056,AB0.053
4,6C0.056,9D0.050,B10.057,450.051,5A0.061,670.051,E90.053,F10.051,700.055,060.057,460.054,B20.053,630.060,AE0.053,210.055,BD0.052


In [2]:
import chipwhisperer.common.api.lascar as cw_lascar
from lascar import *
cw_container = cw_lascar.CWContainer(proj, proj.textouts) #the attack only needs the cipherout, since we neeed to calculate the Hamming Distance between state10(cihper out) and state9(penultimate)
guess_range = range(256)
leakage = cw_lascar.lastround_HD_gen
cpa_engines = [CpaEngine("cpa_%02d" % i, leakage(i), guess_range) for i in range(16)]
session = Session(cw_container, engines=cpa_engines).run(batch_size=1000)

rec_key_10r = []
for i in range(16):
    results = cpa_engines[i].finalize()
    guess = abs(results).max(1).argmax()
    rec_key_10r.append(guess)



2022-01-08 00:28:08,331 - lascar.session - INFO - Session Session: 5000 traces, 18 engines, batch_size=1000, leakage_shape=(129,)
INFO:lascar.session:Session Session: 5000 traces, 18 engines, batch_size=1000, leakage_shape=(129,)
Session |  0%||0 trc/5000 | (18 engines, batch_size=1000, leakage_shape=(129,)) |ETA:  --:--:--

Debug:
1000


Session |100%||5000 trc/5000 | (18 engines, batch_size=1000, leakage_shape=(129,)) |Time:  0:00:14


In [8]:
from chipwhisperer.analyzer.utils.aes_funcs import key_schedule_rounds
# rec_key_10r = results.key_guess() #uncomment this if you use cwa.leakage_models instead of LASCAR
rec_key = key_schedule_rounds(rec_key_10r, 10, 0)
rec_key = bytearray(rec_key)
assert (rec_key == bytearray(proj.keys[0]))
# print(bytearray(proj.keys[0]))

CWbytearray(b'05 42 7c 28 a5 f3 4d f6 8b d5 92 b7 ab 39 08 38')
